In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string as st
import re
import warnings
warnings.filterwarnings("ignore")
import spacy
en = spacy.load('en_core_web_sm')
import string
import os

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stopwords = []
wordnet_lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import keras
from keras.preprocessing.text import Tokenizer

In [4]:
df = pd.read_csv('/content/train - train.csv')

In [5]:
df.head()

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,Milk,0
2,"I get no notification, however the app is real...",notification,0
3,"Love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0


In [6]:
def lower_case(df):
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['text']= df['text'].apply(lambda x: x.lower())
  return df

In [7]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))

In [8]:
lower_case(df)

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,milk,0
2,"i get no notification, however the app is real...",notification,0
3,"love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0
3997,"hi, i m doing for the first time ever and i no...",usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [9]:
conj =['and','but','however']
punctuation_lst =['.','!',',',':',';','?']
conj= conj+punctuation_lst

In [10]:
def conj_based_split(text):
  lst=[]
  lst2 = []
  a=1
  for word in text.split():
    if word not in conj:
      lst.append(word)
    elif (word in conj):
      sent=' '.join(lst)
      lst2.append(sent)
      lst.clear()
    if (a==len(text.split())):
      if(len(lst)):
        sent=' '.join(lst)
        lst2.append(sent)
    a=a+1
  return lst2

In [11]:
def split(data):
  df['new_text']= df['text'].apply(lambda x:conj_based_split(x))

In [12]:
split(df)

In [13]:
df.head(20)

,text,aspect,label,new_text
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it..."
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga..."
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene]
5,"i love notion as a tool, but the mobile just t...",tool,2,"[i love notion as a tool,, the mobile just tak..."
6,unlimited yearly at $216/year,yearly,1,[unlimited yearly at $216/year]
7,"some times missing to deliver, some times diff...",deliver,0,"[some times missing to deliver, some times dif..."
8,would love the ability to “un-delete” deleted ...,tasks,1,[would love the ability to “un-delete” deleted...
9,please add this basic feature,please,2,[please add this basic feature]


In [14]:
def find_aspect(df):
  result=[]
  for index in range(len(df)):
    #print(df['aspect'][index],df['new_text'][index])
    res = [i for i in df['new_text'][index] if df['aspect'][index] in i]
    result.append(res)
  #print(len(result))
  df['new_column'] = result
  return df

In [15]:
find_aspect(df)

,text,aspect,label,new_text,new_column
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?],[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it...",[cannot rely on both milk delivery]
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]","[i get no notification,]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga...",[calendar view were available on iphone!]
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene],[it does not let me load a clip on the scene]
...,...,...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0,[every time i try to edit a page or create a l...,[every time i try to edit a page or create a l...
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0,[unable to load money using wallets (phonepe/o...,[unable to load money using wallets (phonepe/o...
3997,"hi, i m doing for the first time ever and i no...",usage,0,"[hi, i m doing for the first time ever, i not ...",[i not able to understand the usage of all the...
3998,delivery is delayed or cancelled every time i ...,delivery,0,[delivery is delayed or cancelled every time i...,[delivery is delayed or cancelled every time i...


In [16]:
df.head(20)

,text,aspect,label,new_text,new_column
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?],[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it...",[cannot rely on both milk delivery]
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]","[i get no notification,]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga...",[calendar view were available on iphone!]
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene],[it does not let me load a clip on the scene]
5,"i love notion as a tool, but the mobile just t...",tool,2,"[i love notion as a tool,, the mobile just tak...","[i love notion as a tool,]"
6,unlimited yearly at $216/year,yearly,1,[unlimited yearly at $216/year],[unlimited yearly at $216/year]
7,"some times missing to deliver, some times diff...",deliver,0,"[some times missing to deliver, some times dif...","[some times missing to deliver, some times dif..."
8,would love the ability to “un-delete” deleted ...,tasks,1,[would love the ability to “un-delete” deleted...,[would love the ability to “un-delete” deleted...
9,please add this basic feature,please,2,[please add this basic feature],[please add this basic feature]


In [17]:
def convertList(list1):  
    str = ''
    for i in list1: 
        str += i  
    return str  

In [18]:
df['new_column']= df['new_column'].apply(lambda x:convertList(x))

In [19]:
df=df.drop(columns=['new_text'])


In [20]:
df['text'] = df['new_column']


In [21]:
df=df.drop(columns=['new_column'])

In [22]:
df.drop(df.loc[df['label']==1].index, inplace=True)

In [23]:
df

,text,aspect,label
1,cannot rely on both milk delivery,milk,0
2,"i get no notification,",notification,0
4,it does not let me load a clip on the scene,load,0
5,"i love notion as a tool,",tool,2
7,"some times missing to deliver, some times diff...",deliver,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0
3997,i not able to understand the usage of all the ...,usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [24]:
def preprocess(data):
  def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in st.punctuation])
    return punctuationfree
  def tokenize(text):
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]
  def remove_small_words(text):
    return [x for x in text if len(x) > 3 ]
  def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
  def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text 
  def detokenizer(text): 
    str1 = " " 
    return (str1.join(text))
  df['text']= df['text'].apply(lambda x:remove_punctuation(x))
  df['text']= df['text'].apply(lambda x: x.lower())
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))
  df['text'] = df['text'].apply(lambda x : tokenize(x))
  #df['text'] = df['text'].apply(lambda x : remove_small_words(x))
  #df['text'] = df['text'].apply(lambda x:remove_stopwords(x))
  df['text']=df['text'].apply(lambda x:lemmatizer(x))
  df['text']=df['text'].apply(lambda x:detokenizer(x))
  return data

In [25]:
data = preprocess(df)

In [26]:
data

,text,aspect,label
1,cannot rely on both milk delivery,milk,0
2,i get no notification,notification,0
4,it doe not let me load a clip on the scene,load,0
5,i love notion a a tool,tool,2
7,some time missing to deliver some time differe...,deliver,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallet phonepeola m...,unable to load money using wallets,0
3997,i not able to understand the usage of all the ...,usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [27]:
#data[data.text.str.count(' ') > 0]

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x = data['text']
y = data['label']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [30]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [31]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [32]:
model.score(x_test, y_test)

0.7828655834564254

In [33]:
model.predict(vec.transform(['this app could be better']))

array([0])